## Task 2. Import and balance data
Import London addresses. Test the result.

In [2]:
!mongoimport  --host localhost:60000 --db taxiDb --collection londonPostcodes --type csv --file data/London_postcodes.csv  --headerline --ignoreBlanks -v

2022-01-04T16:30:56.555+0200	using write concern: &{majority false 0}
2022-01-04T16:30:56.559+0200	using 8 decoding workers
2022-01-04T16:30:56.559+0200	using 1 insert workers
2022-01-04T16:30:56.563+0200	will listen for SIGTERM, SIGINT, and SIGKILL
2022-01-04T16:30:56.601+0200	filesize: 174255090 bytes
2022-01-04T16:30:56.603+0200	using fields: Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,District Code,Ward Code,Country,County Code,Constituency,Introduced,Terminated,Parish,National Park,Population,Households,Built up area,Built up sub-division,Lower layer super output area,Rural/urban,Region,Altitude,London zone,LSOA Code,Local authority,MSOA Code,Middle layer super output area,Parish Code,Census output area,Constituency Code,Index of Multiple Deprivation,Quality,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income,Sewage Company,Travel To Work Area,ITL level

`mongos> db.stats()`<br>
{
	"raw" : {
		"shard_repl_set2/shard2_node1:27017,shard2_node2:27017,shard2_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 0,
			"views" : 0,
			"objects" : 0,
			"avgObjSize" : 0,
			"dataSize" : 0,
			"storageSize" : 0,
			"totalSize" : 0,
			"indexes" : 0,
			"indexSize" : 0,
			"scaleFactor" : 1,
			"fileSize" : 0,
			"fsUsedSize" : 0,
			"fsTotalSize" : 0,
			"ok" : 1
		},
		"shard_repl_set3/shard3_node1:27017,shard3_node2:27017,shard3_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 1,
			"views" : 0,
			"objects" : 325716,
			"avgObjSize" : 1291.216750174999,
			"dataSize" : 420569955,
			"storageSize" : 64290816,
			"freeStorageSize" : 401408,
			"indexes" : 1,
			"indexSize" : 9809920,
			"indexFreeStorageSize" : 385024,
			"totalSize" : 74100736,
			"totalFreeStorageSize" : 786432,
			"scaleFactor" : 1,
			"fsUsedSize" : 20678135808,
			"fsTotalSize" : 62725623808,
			"ok" : 1
		},
		"shard_repl_set/shard1_node1:27017,shard1_node2:27017,shard1_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 0,
			"views" : 0,
			"objects" : 0,
			"avgObjSize" : 0,
			"dataSize" : 0,
			"storageSize" : 0,
			"totalSize" : 0,
			"indexes" : 0,
			"indexSize" : 0,
			"scaleFactor" : 1,
			"fileSize" : 0,
			"fsUsedSize" : 0,
			"fsTotalSize" : 0,
			"ok" : 1
		},
		"shard_repl_set4/shard4_node1:27017,shard4_node2:27017,shard4_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 0,
			"views" : 0,
			"objects" : 0,
			"avgObjSize" : 0,
			"dataSize" : 0,
			"storageSize" : 0,
			"totalSize" : 0,
			"indexes" : 0,
			"indexSize" : 0,
			"scaleFactor" : 1,
			"fileSize" : 0,
			"fsUsedSize" : 0,
			"fsTotalSize" : 0,
			"ok" : 1
		}
	},
	"objects" : 325716,
	"avgObjSize" : 1291,
	"dataSize" : 420569955,
	"storageSize" : 64290816,
	"totalSize" : 74100736,
	"indexes" : 1,
	"indexSize" : 9809920,
	"scaleFactor" : 1,
	"fileSize" : 0,
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1641308077, 13),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1641308077, 1)
}

### To shard this data following actions need to be performed:
 - go to mongo: `mongo mongodb://localhost:60000`
 - switch to taxiDb: `mongos> use taxiDb`
 - verfity the data is imported to one of the shards: `mongos> db.stats()`
 - enable sharding on db: `mongos> sh.enableSharding("taxiDb")`
 - enable sharding for the newly created collection: `mongos> sh.shardCollection("taxiDb.londonPostcodes", {"_id":1})`

`mongos> sh.enableSharding("taxiDb")`<br>
{
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1641569238, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1641569238, 1)
}
`mongos> sh.shardCollection("taxiDb.londonPostcodes", {"_id":1})`<br>
{
	"collectionsharded" : "taxiDb.londonPostcodes",
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1641569520, 27),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1641569520, 23)
}

### Now we can see that the collection is sharded across 4 replica sets
`mongos> db.printShardingStatus()`<br>
<b>--- Sharding Status ---</b> <br>
  sharding version: {
  	"_id" : 1,
  	"minCompatibleVersion" : 5,
  	"currentVersion" : 6,
  	"clusterId" : ObjectId("61d315727807d7cbd4ec0f68")
  }<br>
  shards:<br>
        {  "_id" : "shard_repl_set",  "host" : "shard_repl_set/shard1_node1:27017,shard1_node2:27017,shard1_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641225152, 2) }
        {  "_id" : "shard_repl_set2",  "host" : "shard_repl_set2/shard2_node1:27017,shard2_node2:27017,shard2_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641227776, 1) }
        {  "_id" : "shard_repl_set3",  "host" : "shard_repl_set3/shard3_node1:27017,shard3_node2:27017,shard3_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641227760, 2) }
        {  "_id" : "shard_repl_set4",  "host" : "shard_repl_set4/shard4_node1:27017,shard4_node2:27017,shard4_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641227797, 1) }<br>
  active mongoses:<br>
        "5.0.5" : 1<br>
  autosplit:<br>
        Currently enabled: yes<br>
  balancer:<br>
        Currently enabled: yes
        Currently running: no
        Collections with active migrations: 
                taxiDb.londonPostcodes started at Fri Jan 07 2022 17:34:22 GMT+0200 (EET)
        Failed balancer rounds in last 5 attempts: 0
        Migration results for the last 24 hours: 
                2 : Success<br>
  databases:<br>
        {  "_id" : "config",  "primary" : "config",  "partitioned" : true }<br>
                config.system.sessions
                        shard key: { "_id" : 1 }
                        unique: false
                        balancing: true
                        chunks:
                                shard_repl_set	256
                                shard_repl_set2	256
                                shard_repl_set3	256
                                shard_repl_set4	256
                        too many chunks to print, use verbose if you want to force print<br>
        {  "_id" : "taxiDb",  "primary" : "shard_repl_set3",  "partitioned" : true,  "version" : {  "uuid" : UUID("e13c509b-a04e-4699-825a-cd083fe4a6fe"),  "timestamp" : Timestamp(1641306656, 1),  "lastMod" : 1 } }<br>
                <b>taxiDb.londonPostcodes</b><br>
                        <b>shard key: { "_id" : 1 }</b><br>
                        unique: false<br>
                        <b>balancing: true<br>
                        chunks:</b><br>
                                shard_repl_set	1
                                shard_repl_set2	1
                                shard_repl_set3	5<br>
                        { "_id" : { "$minKey" : 1 } } -->> { "_id" : ObjectId("61d45a254bd5a323af789cb1") } on : shard_repl_set Timestamp(2, 0)
                        { "_id" : ObjectId("61d45a254bd5a323af789cb1") } -->> { "_id" : ObjectId("61d45a2a4bd5a323af7967bf") } on : shard_repl_set2 Timestamp(3, 0) 
                        { "_id" : ObjectId("61d45a2a4bd5a323af7967bf") } -->> { "_id" : ObjectId("61d45a2f4bd5a323af7a32cd") } on : shard_repl_set3 Timestamp(4, 0) 
                        { "_id" : ObjectId("61d45a2f4bd5a323af7a32cd") } -->> { "_id" : ObjectId("61d45a334bd5a323af7ad897") } on : shard_repl_set3 Timestamp(1, 3) 
                        { "_id" : ObjectId("61d45a334bd5a323af7ad897") } -->> { "_id" : ObjectId("61d45a374bd5a323af7b7e62") } on : shard_repl_set3 Timestamp(1, 4) 
                        { "_id" : ObjectId("61d45a374bd5a323af7b7e62") } -->> { "_id" : ObjectId("61d45a3c4bd5a323af7c242d") } on : shard_repl_set3 Timestamp(1, 5) 
                        { "_id" : ObjectId("61d45a3c4bd5a323af7c242d") } -->> { "_id" : { "$maxKey" : 1 } } on : shard_repl_set3 Timestamp(1, 6) 

## Task 3. Generate more data
Use London addresses for simulations below.
Simulate taxi orders and store in base (with custom intensity).
Simulate taxi movement tracking and store in base (up to 10GB data).
Taxi without order are moving at nearest area.
Simulate taxi drive feedback and store in base.
Test the result.

In [4]:
import pymongo

In [5]:
mongodb_url = "mongodb://localhost:60000"
client = pymongo.MongoClient(mongodb_url)